# Morley Mills Pricing Model (data preparation)

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

### Define function to call rightmove for prices

This function returns a data frame with 1 row per transaction (sale).

In [2]:
def get_properties(postcode, page=1):

    # attempt to pull the prices
    postcode = '-'.join(postcode.split(' ')).lower()
    url = f'https://www.rightmove.co.uk/house-prices/{postcode}.html?page={page}'
    res = requests.get(url)
    print(f'{url} returned {res.status_code}')
    if res.status_code != 200: 
        return pd.DataFrame()

    # attempt to find some transactions
    soup = BeautifulSoup(res.text, 'html.parser')
    s = soup.find_all('script')[-2]
    d = json.loads(s.string[s.string.find('{'):])
    df = pd.DataFrame(d['results']['properties'])
    if df.empty: 
        return pd.DataFrame()

    # melt to make 1 row per transaction
    df = df.drop('transactions', axis=1).join(df.transactions.explode()).reset_index(drop=True)
    df = df.drop('transactions', axis=1).join(pd.json_normalize(df.transactions))
    return df

### Iterate over postcodes of intest and compile results in a single frame

In [3]:
dflist = []

# define list of postcodes within area
for postcode in ['NG5 6JL']:
    page = 1
    while True:
        time.sleep(1)
        df = get_properties(postcode, page)
        if df.empty: break
        dflist.append(df)
        page += 1

https://www.rightmove.co.uk/house-prices/ng5-6jl.html?page=1 returned 200
https://www.rightmove.co.uk/house-prices/ng5-6jl.html?page=2 returned 200
https://www.rightmove.co.uk/house-prices/ng5-6jl.html?page=3 returned 200
https://www.rightmove.co.uk/house-prices/ng5-6jl.html?page=4 returned 200


In [4]:
df = pd.concat(dflist).reset_index(drop=True)

In [5]:
df['numericPrice'] = pd.to_numeric(df.displayPrice.str.replace('£', '').str.replace(',', ''))

### Expand address field into categories

In [6]:
df_address = df.address.str.split(', ').explode().to_frame()

In [7]:
# extract the postcode to a new fields and remove it from the original one
postcode_regex = r'[A-Z]{1,2}[0-9]{1,2}[ ]{0,1}[0-9]{1,2}[A-Z]{1,2}$'
df_address['postcode'] = df_address.address.str.extract(f'({postcode_regex})')
df_address.address = df_address.apply(
    lambda x: x.address if isinstance(x.postcode, float) else x.address.replace(x.postcode, ''), axis=1)

In [8]:
# extract the street number to a new fields and remove it from the original one
df_address['number'] = pd.to_numeric(df_address.address.str.replace(r'[^0-9]*$', '', regex=True), errors='coerce')
df_address['number'] = df_address.apply(
    lambda x: x.address if x.number > 0 else None, axis=1)

In [9]:
# indentify any prefixes or house names and extract those
df_address['prefix'] = df_address.number.notna().groupby(level=0).cumsum() == 0
df_address['prefix'] = df_address.apply(
    lambda x: x.address if x.prefix else None, axis=1)

In [10]:
df_address.loc[df_address.notna().sum(axis=1) > 1, 'address'] = None

In [11]:
# number the remaining fields address_1, address_2 etc.
address_to_idx = [None] + df_address.address.dropna().drop_duplicates().tolist()
df_address['counter'] = df_address.address.apply(lambda x: address_to_idx.index(x)).replace({0: None})
df_address['counter'] = df_address.counter.groupby(level=0).rank(method='first').convert_dtypes()

In [12]:
# melt and repivot to get back to original shape
df_address_melt = df_address.reset_index().melt(id_vars=['index', 'counter']).dropna(subset='value')
df_address_melt['column'] = (df_address_melt.variable + '_' + df_address_melt.counter.astype(str)).str.replace('_<NA>', '')
df_address = df_address_melt.pivot(index='index', columns='column', values='value')

### Append the individual address fields to the original dataset and save

In [13]:
df = df.join(df_address)

In [14]:
df.to_csv('moreley_mills_prices.csv', index=False)

In [15]:
df

,address,propertyType,images,hasFloorPlan,location,detailUrl,displayPrice,dateSold,tenure,newBuild,numericPrice,address_1,address_2,address_3,address_4,number,postcode
0,"8, Morley Mills, Morley Street, Daybrook, Nott...",Flat,{'imageUrl': '/spw/images/placeholder/no-image...,False,"{'lat': 52.99703, 'lng': -1.13713}",https://www.rightmove.co.uk/house-prices/detai...,"£191,000",15 Jul 2024,Leasehold,False,191000,Morley Mills,Morley Street,Daybrook,Nottingham,8,NG5 6JL
1,"8, Morley Mills, Morley Street, Daybrook, Nott...",Flat,{'imageUrl': '/spw/images/placeholder/no-image...,False,"{'lat': 52.99703, 'lng': -1.13713}",https://www.rightmove.co.uk/house-prices/detai...,"£156,000",12 Apr 2019,Leasehold,False,156000,Morley Mills,Morley Street,Daybrook,Nottingham,8,NG5 6JL
2,"8, Morley Mills, Morley Street, Daybrook, Nott...",Flat,{'imageUrl': '/spw/images/placeholder/no-image...,False,"{'lat': 52.99703, 'lng': -1.13713}",https://www.rightmove.co.uk/house-prices/detai...,"£140,000",14 Aug 2015,Leasehold,False,140000,Morley Mills,Morley Street,Daybrook,Nottingham,8,NG5 6JL
3,"8, Morley Mills, Morley Street, Daybrook, Nott...",Flat,{'imageUrl': '/spw/images/placeholder/no-image...,False,"{'lat': 52.99703, 'lng': -1.13713}",https://www.rightmove.co.uk/house-prices/detai...,"£135,730",2 Aug 2013,Leasehold,False,135730,Morley Mills,Morley Street,Daybrook,Nottingham,8,NG5 6JL
4,"8, Morley Mills, Morley Street, Daybrook, Nott...",Flat,{'imageUrl': '/spw/images/placeholder/no-image...,False,"{'lat': 52.99703, 'lng': -1.13713}",https://www.rightmove.co.uk/house-prices/detai...,"£110,000",21 Jun 2001,Leasehold,False,110000,Morley Mills,Morley Street,Daybrook,Nottingham,8,NG5 6JL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,"47, Morley Mills, Morley Street, Daybrook, Not...",Flat,{'imageUrl': '/spw/images/placeholder/no-image...,False,"{'lat': 52.99703, 'lng': -1.13713}",https://www.rightmove.co.uk/house-prices/detai...,"£80,000",14 Sep 2001,Leasehold,False,80000,Morley Mills,Morley Street,Daybrook,Nottingham,47,NG5 6JL
191,"20, Morley Mills, Morley Street, Daybrook, Not...",Flat,{'imageUrl': '/spw/images/placeholder/no-image...,False,"{'lat': 52.99703, 'lng': -1.13713}",https://www.rightmove.co.uk/house-prices/detai...,"£75,000",29 Jun 2001,Leasehold,True,75000,Morley Mills,Morley Street,Daybrook,Nottingham,20,NG5 6JL
192,"17, Morley Mills, Morley Street, Daybrook, Not...",Flat,{'imageUrl': '/spw/images/placeholder/no-image...,False,"{'lat': 52.99703, 'lng': -1.13713}",https://www.rightmove.co.uk/house-prices/detai...,"£130,000",29 Jun 2001,Leasehold,False,130000,Morley Mills,Morley Street,Daybrook,Nottingham,17,NG5 6JL
193,"29, Morley Mills, Morley Street, Daybrook, Not...",Flat,{'imageUrl': '/spw/images/placeholder/no-image...,False,"{'lat': 52.99703, 'lng': -1.13713}",https://www.rightmove.co.uk/house-prices/detai...,"£120,000",22 Jun 2001,Leasehold,False,120000,Morley Mills,Morley Street,Daybrook,Nottingham,29,NG5 6JL
